### Importación de archivos y librerías

In [1]:
! pip install sqlalchemy==1.4.46
! pip install pandasql==0.7.3
! pip install xgboost


[notice] A new release of pip available: 22.2.2 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.2.2 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.2.2 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import sys
sys.path.append("../")
from sales import data_holder, data_info, common, feature_engineering
import pandas as pd
import pandasql as ps

### Generación de dataframes

In [3]:
data = pd.read_csv('../data/data sales csv/sales_data.csv', delimiter=',')
data_holder.set_data(data)
data_info.preview()

Data shape:  (1162971, 29)


,FechaFactura,mes,business_unit,team,category,class,gender,tipo_producto,sku,cc,...,CURRENCYCODE,AfectoFinal,CantVendida,Promedio de Mg,product_name,familia(collection),Rut_Cliente,Año,Semana,Season_fecha
0,08-02-2023,2,NaN,NaN,NaN,NaN,NaN,NaN,DESPACHO,NaN,...,CLP,$2.479,1,100%,NaN,NaN,10530209-6,2023,6,SP22
1,08-02-2023,2,NaN,NaN,NaN,NaN,NaN,NaN,DESPACHO,NaN,...,CLP,$0,1,NaN,NaN,NaN,13620253-7,2023,6,SP22
2,08-02-2023,2,NaN,NaN,NaN,NaN,NaN,NaN,DESPACHO,NaN,...,CLP,$0,1,NaN,NaN,NaN,15170047-0,2023,6,SP22
3,08-02-2023,2,NaN,NaN,NaN,NaN,NaN,NaN,DESPACHO,NaN,...,CLP,$0,1,NaN,NaN,NaN,15327358-8,2023,6,SP22
4,08-02-2023,2,NaN,NaN,NaN,NaN,NaN,NaN,DESPACHO,NaN,...,CLP,$3.319,1,100%,NaN,NaN,16711666-3,2023,6,SP22


In [4]:
data.columns

Index(['FechaFactura', 'mes', 'business_unit', 'team', 'category', 'class',
       'gender', 'tipo_producto', 'sku', 'cc', 'itemnumber', 'color_id',
       'color_name', 'size', 'season_cl', 'Canal', 'TiendaDim', 'Dimensión',
       'CostoTotal', 'CURRENCYCODE', 'AfectoFinal', 'CantVendida',
       'Promedio de Mg', 'product_name', 'familia(collection)', 'Rut_Cliente',
       'Año', 'Semana', 'Season_fecha'],
      dtype='object')

In [5]:
data['Season_fecha'].unique()

array(['SP22', 'FA21', 'SP21', 'FA20', 'SP20', 'FA19', 'SP19', 'FA18',
       'SP18', 'FA17'], dtype=object)

In [6]:
data['familia(collection)'].unique()

array([nan, 'OTROS', 'TORRENTSHELL', 'DOWN SWEATER', 'TECHFACE',
       'TRUCKER HAT', 'ALPINE ICON', 'P-6', 'BACK FOR GOOD', 'BLACK HOLE',
       'DIRT ROAMER', 'CAP MW', 'GI', 'BAGGIES', 'STAND UP', 'INTERSTATE',
       'NANO PUFF', 'CAP COOL DAILY', 'RETRO PILE', 'MICRO D', 'COZY',
       'SYNCHILLA', 'FURRY FRIENDS', 'BETTER SWEATER', 'ROC', 'ALL-WEAR',
       'FJORD FLANNEL', 'BACK STEP', 'SILENT', 'RETRO-X', 'AHNYA',
       'LOS GATOS', 'ALL SEASONS', 'FARRIER', 'VENGA', 'MAIPO',
       'MICRO PUFF', 'NANO-AIR', 'R1 AIR', 'R1 DAILY', 'SNOWBELLE',
       'RESPONSIBILI-TEE', 'ORGANIC T-SHIRT', 'ORGANIC COTTON', 'ROPC',
       'MERINO', 'RE-TOOL', 'REFUGIO', 'ASCENSIONIST', 'ATOM', 'ARBOR',
       'ALTVIA', 'GUIDEWATER', 'SUN STRETCH', 'SOL PATROL',
       'SWIFTCURRENT', 'QUANDARY', 'CAP TW', 'SELF GUIDED', 'CAP AIR',
       'CAP COOL TRAIL', 'NINE TRAILS', 'STRIDER', 'HOUDINI', 'BIVY',
       'DOWNDRIFT', 'HAPPY HIKE', 'MICRODINI', 'HAMPI', 'TRIOLET',
       'ALPLIGHT', 'CALCITE',

In [7]:
df_warehouses = pd.read_csv('../data/data warehouses/warehouses.csv', delimiter=';')
df_warehouses.head()

,warehouse,is_metropolitan_zone,is_inside_mall,Unnamed: 3
0,LASCONDES,1,1,NaN
1,COSTANERA,1,1,NaN
2,LADEHESA,1,1,NaN
3,MALLSPORT,1,1,NaN
4,PTOVARAS,0,0,NaN


In [8]:
df_sale_campaigns = pd.read_csv('../data/data sale campaigns/sale_campaigns.csv', delimiter=';', encoding='latin-1')
df_sale_campaigns = df_sale_campaigns.rename(columns={
    'Fecha Inicio': 'start_date',
    'Fecha Fin': 'end_date',
    'Descripción': 'campaign_name'
})
df_sale_campaigns.head()

,start_date,end_date,campaign_name
0,01-03-2019,17-03-2019,SALE
1,01-05-2019,12-05-2019,DIA DE LA MADRE
2,05-06-2019,16-06-2019,DIA DEL PADRE
3,30-08-2019,23-09-2019,SALE
4,21-02-2020,15-03-2020,SALE


### Data preprocessing

In [9]:
data_info.duplicates()
data_holder.delete_duplicates()
data_info.duplicates()

Duplicates in Data set: 2602,(0.22%)
Duplicates in Data set: 0,(0.0%)


We remove all the items that refer to the shipping charge

In [10]:
data_holder.remove_rows_by_values(['sku'], ['DESPACHO'])
data_holder.remove_rows_by_values(['sku'], ['ENVIO'])
data_info.preview()

Data shape:  (1075322, 29)


,FechaFactura,mes,business_unit,team,category,class,gender,tipo_producto,sku,cc,...,CURRENCYCODE,AfectoFinal,CantVendida,Promedio de Mg,product_name,familia(collection),Rut_Cliente,Año,Semana,Season_fecha
16,08-02-2023,2,LIFE OUTDOORS,M S SPORTSWEAR,TOPS,S/S TOPS,MENS,CAMISAS,42245CYPIM,42245CYPI,...,CLP,$32.773,1,68%,M S DAILY SHIRT,OTROS,19061589-8,2023,6,SP22
17,08-02-2023,2,LIFE OUTDOORS,W S SPORTSWEAR,BOTTOMS,PANTS,WOMENS,PANTALONES,56566WWIBS,56566WWIB,...,CLP,$41.176,1,69%,W S GARDEN ISLAND PANTS,OTROS,9194419-7,2023,6,SP22
18,08-02-2023,2,MOUNTAINS,ALPINE,OUTERWEAR,OUTERWEAR JACKETS,MENS,IMPERMEABLES,85240BLKM,85240BLK,...,CLP,$104.202,1,64%,M S TORRENTSHELL 3L JKT,TORRENTSHELL,16923253-9,2023,6,SP22
19,08-02-2023,2,MOUNTAINS,ALPINE,OUTERWEAR,OUTERWEAR JACKETS,MENS,PARKAS,84701CNYL,84701CNY,...,CLP,$60.252,1,3%,M S DOWN SWEATER HOODY,DOWN SWEATER,19133957-6,2023,6,SP22
20,08-02-2023,2,MOUNTAINS,ALPINE,TOPS,MIDLAYER JACKETS,MENS,CHAQUETAS,83580BLKL,83580BLK,...,CLP,$91.597,1,65%,M S R1 TECHFACE JKT,TECHFACE,16711666-3,2023,6,SP22


In [11]:
df = data_holder.get_data()
print(df[df['CantVendida'] <= 0].shape)
df = df[df['CantVendida'] > 0]
data_holder.set_data(df)

(92216, 29)


In [12]:
data_holder.remove_columns_by_features([
       'FechaFactura', 'sku', 'cc', 'itemnumber', 'TiendaDim',
       'CostoTotal', 'AfectoFinal', 'CantVendida'
])
data_info.preview()

Data shape:  (983106, 8)


,FechaFactura,sku,cc,itemnumber,TiendaDim,CostoTotal,AfectoFinal,CantVendida
16,08-02-2023,42245CYPIM,42245CYPI,42245,Ecommerce,$10.496,$32.773,1
17,08-02-2023,56566WWIBS,56566WWIB,56566,Ecommerce,$12.740,$41.176,1
18,08-02-2023,85240BLKM,85240BLK,85240,Ecommerce,$37.794,$104.202,1
19,08-02-2023,84701CNYL,84701CNY,84701,Ecommerce,$58.591,$60.252,1
20,08-02-2023,83580BLKL,83580BLK,83580,Ecommerce,$32.175,$91.597,1


In [13]:
data_holder.rename_columns({
                    'FechaFactura': 'date',
                    'TiendaDim': 'warehouse',
                    'CostoTotal': 'cost',
                    'AfectoFinal': 'sale_amount', 
                    'CantVendida': 'quantity',
                    })
data_info.preview()

Data shape:  (983106, 8)


,date,sku,cc,itemnumber,warehouse,cost,sale_amount,quantity
16,08-02-2023,42245CYPIM,42245CYPI,42245,Ecommerce,$10.496,$32.773,1
17,08-02-2023,56566WWIBS,56566WWIB,56566,Ecommerce,$12.740,$41.176,1
18,08-02-2023,85240BLKM,85240BLK,85240,Ecommerce,$37.794,$104.202,1
19,08-02-2023,84701CNYL,84701CNY,84701,Ecommerce,$58.591,$60.252,1
20,08-02-2023,83580BLKL,83580BLK,83580,Ecommerce,$32.175,$91.597,1


In [14]:
data_info.get_types()

Data types: 

date           object
sku            object
cc             object
itemnumber     object
warehouse      object
cost           object
sale_amount    object
quantity        int64
dtype: object


In [15]:
df = data_holder.get_data()   
df['date'] = pd.to_datetime(df['date'], format='%d-%m-%Y')
df['sale_amount'] = df['sale_amount'].str.replace('[$.]', '').astype(float).astype(int)
df['sale_amount'] = df['sale_amount'].astype(int)
df['cost'] = df['cost'].str.replace('[$.]', '').astype(float).astype(int)
df['cost'] = df['cost'].astype(int)
df['quantity'] = df['quantity'].astype(int)

data_holder.set_data(df)
df.head()

C:\Users\Enrique Urrutia\AppData\Local\Temp\ipykernel_15748\1163990109.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df['sale_amount'] = df['sale_amount'].str.replace('[$.]', '').astype(float).astype(int)
C:\Users\Enrique Urrutia\AppData\Local\Temp\ipykernel_15748\1163990109.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  df['cost'] = df['cost'].str.replace('[$.]', '').astype(float).astype(int)


,date,sku,cc,itemnumber,warehouse,cost,sale_amount,quantity
16,2023-02-08,42245CYPIM,42245CYPI,42245,Ecommerce,10496,32773,1
17,2023-02-08,56566WWIBS,56566WWIB,56566,Ecommerce,12740,41176,1
18,2023-02-08,85240BLKM,85240BLK,85240,Ecommerce,37794,104202,1
19,2023-02-08,84701CNYL,84701CNY,84701,Ecommerce,58591,60252,1
20,2023-02-08,83580BLKL,83580BLK,83580,Ecommerce,32175,91597,1


Delete rows that refer to *Pop Up* warehouse 

In [16]:
df = data_holder.get_data()
print(df[df['warehouse'] == 'Pop UP'].shape)
df = df[df['warehouse'] != 'Pop UP']
data_holder.set_data(df)

(39696, 8)


Standardize *warehouses* names

In [17]:
data_holder.get_data()['warehouse'].unique()

array(['Ecommerce', 'Concepción', 'Puerto Varas', 'Temuco', 'Alerce',
       'Pucón', 'Coyhaique', 'Costanera', 'La Dehesa', 'Mall Sport',
       'Osorno', 'BuenaVentura', 'Wholesale', 'Alto Las Condes',
       'El Golf'], dtype=object)

In [18]:
warehouse_names = {
    'Alto Las Condes': 'LASCONDES', 'Mall Sport': 'MALLSPORT',
    'La Dehesa': 'LADEHESA', 'Pucón': 'PUCON', 'Temuco': 'TEMUCO',
    'Concepción': 'CONCEPCION', 'Coyhaique': 'COYHAIQUE',
    'Puerto Varas': 'PTOVARAS', 'BuenaVentura': 'BUENAVENTURA',
    'Wholesale': ' WHOLESALE', 'Ecommerce': 'ECOMMERCE',
    'Costanera': 'COSTANERA', 'El Golf': 'ELGOLF',
    'Alerce': 'ALERCE', 'Osorno': 'OSORNO'
}
df['warehouse'] = df['warehouse'].apply(lambda x: warehouse_names[x])

data_holder.set_data(df)
df['warehouse'].unique()

array(['ECOMMERCE', 'CONCEPCION', 'PTOVARAS', 'TEMUCO', 'ALERCE', 'PUCON',
       'COYHAIQUE', 'COSTANERA', 'LADEHESA', 'MALLSPORT', 'OSORNO',
       'BUENAVENTURA', ' WHOLESALE', 'LASCONDES', 'ELGOLF'], dtype=object)

#### Codificate to unit price and group data for date-sku-warehouse

In [19]:
data_holder.set_data(
    feature_engineering.add_unit_price_information(data_holder.get_data())
)
data_info.preview()

Data shape:  (943410, 9)


,date,sku,cc,itemnumber,warehouse,cost,sale_amount,quantity,unit_price
0,2023-02-08,42245CYPIM,42245CYPI,42245,ECOMMERCE,10496,32773,1,32773
1,2023-02-08,56566WWIBS,56566WWIB,56566,ECOMMERCE,12740,41176,1,41176
2,2023-02-08,85240BLKM,85240BLK,85240,ECOMMERCE,37794,104202,1,104202
3,2023-02-08,84701CNYL,84701CNY,84701,ECOMMERCE,58591,60252,1,60252
4,2023-02-08,83580BLKL,83580BLK,83580,ECOMMERCE,32175,91597,1,91597


In [20]:
data_holder.group_data(
        ['date', 'sku', 'cc', 'itemnumber', 'warehouse'],
        {'cost' : 'mean',
         'unit_price': 'mean',
         'sale_amount' : 'max',
         'quantity' : 'sum'})
data_info.preview()

Data shape:  (876141, 9)


,date,sku,cc,itemnumber,warehouse,cost,unit_price,sale_amount,quantity
0,2018-08-02,27810NUVGM,27810NUVG,27810,LASCONDES,0.0,8739.0,8739,1
1,2018-08-02,28322FYBBL,28322FYBB,28322,LASCONDES,0.0,13782.0,13782,1
2,2018-08-02,44422BLKXL,44422BLK,44422,LASCONDES,0.0,8739.0,8739,1
3,2018-08-03,12010ZOOALL,12010ZOO,12010,MALLSPORT,72588.0,50084.0,50084,1
4,2018-08-03,12011ZOOALL,12011ZOO,12011,LADEHESA,81820.0,56807.0,56807,1


In [21]:
data_holder.get_data().to_csv('../data/temp/base_processed_data.csv')

### Feature engineering

#### Add *Date* relate information

In [18]:
print("#### Adding Date related info")
df = data_holder.get_data()

df = feature_engineering.add_weekday_information(df)
print("Ok feature 1")
df = feature_engineering.add_month_information(df)
print("Ok feature 2")
df = feature_engineering.add_year_information(df)
print("Ok feature 3")
df = feature_engineering.add_offer_day_information(df, df_sale_campaigns)
print("Ok feature 4")

data_holder.set_data(df)
data_info.preview()

#### Adding Date related info
Ok feature 1
Ok feature 2
Ok feature 3
Ok feature 4
Data shape:  (2628423, 13)


,date,sku,cc,itemnumber,warehouse,cost,unit_price,sale_amount,quantity,weekday,month,year,is_offer_day
0,2018-08-02,27810NUVGM,27810NUVG,27810,LASCONDES,0.0,8739.0,8739,1,4,8,2018,1
1,2018-08-02,27810NUVGM,27810NUVG,27810,LASCONDES,0.0,8739.0,8739,1,4,8,2018,1
2,2018-08-02,27810NUVGM,27810NUVG,27810,LASCONDES,0.0,8739.0,8739,1,4,8,2018,1
3,2018-08-02,28322FYBBL,28322FYBB,28322,LASCONDES,0.0,13782.0,13782,1,4,8,2018,1
4,2018-08-02,28322FYBBL,28322FYBB,28322,LASCONDES,0.0,13782.0,13782,1,4,8,2018,1


#### Add *Warehouse* relate information

In [19]:
warehouse_dummies = pd.get_dummies(df['warehouse'], prefix='warehouse')
warehouse_dummies

,warehouse_ WHOLESALE,warehouse_ALERCE,warehouse_BUENAVENTURA,warehouse_CONCEPCION,warehouse_COSTANERA,warehouse_COYHAIQUE,warehouse_ECOMMERCE,warehouse_ELGOLF,warehouse_LADEHESA,warehouse_LASCONDES,warehouse_MALLSPORT,warehouse_OSORNO,warehouse_PTOVARAS,warehouse_PUCON,warehouse_TEMUCO
0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2628418,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
2628419,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
2628420,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
2628421,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0


In [20]:
print("#### Adding Warehouse related info")
df = data_holder.get_data()

df = feature_engineering.add_warehouse_is_metropolitan_zone(df, df_warehouses)
print("Ok feature 1")
df = feature_engineering.add_warehouse_is_inside_mall(df, df_warehouses)
print("Ok feature 2")

data_holder.set_data(df)
data_info.preview()

#### Adding Warehouse related info
Ok feature 1
Ok feature 2
Data shape:  (2628423, 15)


,date,sku,cc,itemnumber,warehouse,cost,unit_price,sale_amount,quantity,weekday,month,year,is_offer_day,is_metropolitan_zone,is_inside_mall
0,2018-08-02,27810NUVGM,27810NUVG,27810,LASCONDES,0.0,8739.0,8739,1,4,8,2018,1,1,1
1,2018-08-02,27810NUVGM,27810NUVG,27810,LASCONDES,0.0,8739.0,8739,1,4,8,2018,1,1,1
2,2018-08-02,27810NUVGM,27810NUVG,27810,LASCONDES,0.0,8739.0,8739,1,4,8,2018,1,1,1
3,2018-08-02,28322FYBBL,28322FYBB,28322,LASCONDES,0.0,13782.0,13782,1,4,8,2018,1,1,1
4,2018-08-02,28322FYBBL,28322FYBB,28322,LASCONDES,0.0,13782.0,13782,1,4,8,2018,1,1,1


#### Add *SKU - Date*  related info

In [21]:
print("#### Adding Sku-date related info")
df = data_holder.get_data()

df = feature_engineering.add_sku_cumulative_sales_in_the_week(df)
print("Ok feature 1")
df = feature_engineering.add_sku_cumulative_sales_in_the_month(df)
print("Ok feature 2")
df = feature_engineering.add_sku_cumulative_sales_in_the_year(df)
print("Ok feature 3")
df = feature_engineering.add_sku_historic_sales(df)
print("Ok feature 4")
df = feature_engineering.add_sku_historic_sales_same_day_of_the_week(df)
print("Ok feature 5")
df = feature_engineering.add_sku_historic_sales_same_month(df)
print("Ok feature 6")

data_holder.set_data(df)
data_info.preview()

#### Adding Sku-date related info
Ok feature 1
Ok feature 2
Ok feature 3
Ok feature 4
Ok feature 5
Ok feature 6
Data shape:  (2628423, 21)


,date,sku,cc,itemnumber,warehouse,cost,unit_price,sale_amount,quantity,weekday,...,year,is_offer_day,is_metropolitan_zone,is_inside_mall,sku_cumulative_sales_in_the_week,sku_cumulative_sales_in_the_month,sku_cumulative_sales_in_the_year,sku_historic_sales,sku_historic_sales_same_day_of_the_week,sku_historic_sales_same_month
0,2021-12-05,11959HEXGALL,11959HEXG,11959,MALLSPORT,4123.0,14118.0,14118,1,0,...,2021,1,1,1,0,0,6,6,0,0
1,2021-12-05,11959HEXGALL,11959HEXG,11959,MALLSPORT,4123.0,14118.0,14118,1,0,...,2021,1,1,1,0,0,6,6,1,0
2,2021-12-05,11959HEXGALL,11959HEXG,11959,MALLSPORT,4123.0,14118.0,14118,1,0,...,2021,1,1,1,0,0,6,6,2,0
3,2022-02-21,11959HEXGALL,11959HEXG,11959,MALLSPORT,4124.0,15966.0,15966,1,1,...,2022,1,1,1,0,15,27,39,0,0
4,2022-02-21,11959HEXGALL,11959HEXG,11959,MALLSPORT,4124.0,15966.0,15966,1,1,...,2022,1,1,1,0,15,27,39,1,0


#### Add *SKU-Warehouse-Date* related info

In [22]:
print("#### Adding Sku-warehouse-date related info")
df = data_holder.get_data()

df = feature_engineering.add_sku_warehouse_last_xdays_sales(df, 7)
print("Ok feature 1")
df = feature_engineering.add_sku_warehouse_last_xdays_mean_sales(df, 7)
print("Ok feature 2")
df = feature_engineering.add_sku_warehouse_last_xdays_sales(df, 10)
print("Ok feature 3")
df = feature_engineering.add_sku_warehouse_last_xdays_mean_sales(df, 10)
print("Ok feature 4")
df = feature_engineering.add_sku_warehouse_last_xdays_sales(df, 14)
print("Ok feature 5")
df = feature_engineering.add_sku_warehouse_last_xdays_mean_sales(df, 14)
print("Ok feature 6")
df = feature_engineering.add_sku_warehouse_last_xdays_sales(df, 30)
print("Ok feature 7")
df = feature_engineering.add_sku_warehouse_last_xdays_mean_sales(df, 30)
print("Ok feature 8")

df = feature_engineering.add_sku_warehouse_historic_sales(df)
print("Ok feature 9")
df = feature_engineering.add_sku_warehouse_historic_sales_same_day_of_the_week(df)
print("Ok feature 10")
df = feature_engineering.add_sku_warehouse_historic_sales_same_month(df)
print("Ok feature 11")

df = feature_engineering.add_sku_warehouse_cumulative_sales_in_the_week(df)
print("Ok feature 10")
df = feature_engineering.add_sku_warehouse_cumulative_sales_in_the_month(df)
print("Ok feature 11")
df = feature_engineering.add_sku_warehouse_cumulative_sales_in_the_year(df)
print("Ok feature 12")

df = feature_engineering.add_y_sku_warehouse_next_xdays_sales(df, 7)
print("Ok feature 13")
df = feature_engineering.add_y_sku_warehouse_next_xdays_sales(df, 14)
print("Ok feature 14")
df = feature_engineering.add_y_sku_warehouse_next_xdays_sales(df, 30)
print("Ok feature 15")


data_holder.set_data(df)
data_info.preview()

#### Adding Sku-warehouse-date related info
Ok feature 1
Ok feature 2
Ok feature 3
Ok feature 4
Ok feature 5
Ok feature 6
Ok feature 7
Ok feature 8
Ok feature 9
Ok feature 10
Ok feature 11
Ok feature 10
Ok feature 11
Ok feature 12
Ok feature 13
Ok feature 14
Ok feature 15
Data shape:  (876141, 38)


,date,sku,cc,itemnumber,warehouse,cost,unit_price,sale_amount,quantity,weekday,...,sku_warehouse_last_30days_mean_sales,sku_warehouse_historic_sales,sku_warehouse_historic_sales_same_day_of_the_week,sku_warehouse_historic_sales_same_month,sku_warehouse_cumulative_sales_in_the_week,sku_warehouse_cumulative_sales_in_the_month,sku_warehouse_cumulative_sales_in_the_year,y_sku_warehouse_next_7days_sales,y_sku_warehouse_next_14days_sales,y_sku_warehouse_next_30days_sales
0,2018-08-02,27810NUVGM,27810NUVG,27810,LASCONDES,0.0,8739.0,8739,1,4,...,0.0,0,0,0,0,0,0,0,0,0
1,2018-08-02,28322FYBBL,28322FYBB,28322,LASCONDES,0.0,13782.0,13782,1,4,...,0.0,0,0,0,0,0,0,0,0,0
2,2018-08-02,44422BLKXL,44422BLK,44422,LASCONDES,0.0,8739.0,8739,1,4,...,0.0,0,0,0,0,0,0,0,0,0
3,2018-08-03,12010ZOOALL,12010ZOO,12010,MALLSPORT,72588.0,50084.0,50084,1,5,...,0.0,0,0,0,0,0,0,0,0,0
4,2018-08-03,12011ZOOALL,12011ZOO,12011,LADEHESA,81820.0,56807.0,56807,1,5,...,0.0,0,0,0,0,0,0,0,0,0


Saving the processed data

In [23]:
data_holder.get_data().to_csv('../data/temp/processed_data.csv')